In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pd_train = pd.read_csv('/content/drive/MyDrive/Datasets/fashion-mnist_train.csv')

In [4]:
pd_test = pd.read_csv('/content/drive/MyDrive/Datasets/fashion-mnist_test.csv')

In [5]:
pd_train.shape

(60000, 785)

In [6]:
pd_test.shape

(10000, 785)

In [7]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_der(x):
    return sigmoid(x) *(1-sigmoid (x))

def softmax(A):
    expA = np.exp(A)
    return expA / expA.sum(axis=1, keepdims=True)
def stable_softmax(X):
    exps = np.exp(X - np.max(X))
    return exps / np.sum(exps)

In [16]:
x_train =np.array(pd_train.drop('label',axis=1))/255

In [17]:
x_train.shape

(60000, 784)

In [10]:
y_train = np.array(pd_train['label'])

In [11]:
y = np.zeros((60000,10))

In [12]:
for i in range(60000):
  classs = y_train[i]
  y[i][classs] = 1

In [13]:
y_train = y
y_train.shape

(60000, 10)

In [14]:
x = 24
print(f'hi{x}')

hi24


In [27]:
instances = x_train.shape[0]
attributes = x_train.shape[1]
hidden_nodes = 1568
output_labels = 10

wh = np.random.rand(attributes,hidden_nodes)
bh = np.random.randn(hidden_nodes)

wo = np.random.rand(hidden_nodes,output_labels)
bo = np.random.randn(output_labels)
lr = 0.01

error_cost = []

for epoch in range(70):
############# feedforward

    # Phase 1
    zh = np.dot(x_train, wh) + bh
    ah = sigmoid(zh)

    # Phase 2
    zo = np.dot(ah, wo) + bo
    ao = stable_softmax(zo)

########## Back Propagation

########## Phase 1

    dcost_dzo = ao - y_train
    dzo_dwo = ah

    dcost_wo = np.dot(dzo_dwo.T, dcost_dzo)

    dcost_bo = dcost_dzo

########## Phases 2

    dzo_dah = wo
    dcost_dah = np.dot(dcost_dzo , dzo_dah.T)
    dah_dzh = sigmoid_der(zh)
    dzh_dwh = x_train
    dcost_wh = np.dot(dzh_dwh.T, dah_dzh * dcost_dah)

    dcost_bh = dcost_dah * dah_dzh

    # Update Weights ================

    wh -= lr * dcost_wh
    bh -= lr * dcost_bh.sum(axis=0)

    wo -= lr * dcost_wo
    bo -= lr * dcost_bo.sum(axis=0)
    print(f'epoch {epoch}')

    
    loss = np.sum(-y_train * np.log(ao))
    print('Loss function value: ', loss)
    error_cost.append(loss)

epoch 0
Loss function value:  1639282.864445869
epoch 1
Loss function value:  1541785.140693523
epoch 2
Loss function value:  1415684.0814017565
epoch 3
Loss function value:  1326418.2067486786
epoch 4
Loss function value:  1238101.6264355602
epoch 5
Loss function value:  1097042.9420964036
epoch 6
Loss function value:  987204.1270232869
epoch 7
Loss function value:  891348.9065194365
epoch 8
Loss function value:  856873.1936918487
epoch 9
Loss function value:  818643.577958346
epoch 10
Loss function value:  804217.6028409855
epoch 11
Loss function value:  799133.9544431493
epoch 12
Loss function value:  798560.2848206708
epoch 13
Loss function value:  798364.9107361959
epoch 14
Loss function value:  798308.784335375
epoch 15
Loss function value:  798289.8984057176
epoch 16
Loss function value:  798283.9691904394
epoch 17
Loss function value:  798282.0213072529
epoch 18
Loss function value:  798281.3964553224
epoch 19
Loss function value:  798281.1931483353
epoch 20
Loss function value

In [31]:
x_test = np.array(pd_test.drop('label',axis=1))/255

In [32]:
y2 = np.array(pd_test['label'])

In [33]:
x_test.shape

(10000, 784)

In [34]:
y_test = np.zeros((10000,10))

In [35]:
for i in range(10000):
  classs = y2[i]
  y_test[i][classs] = 1

In [36]:
    zh1 = np.dot(x_test, wh) + bh
    ah1 = sigmoid(zh1)

    # Phase 2
    zo1 = np.dot(ah1, wo) + bo
    ao1 = stable_softmax(zo1)

In [51]:
ao1.shape

(10000, 10)

In [52]:
ao1

array([[1.00069370e-05, 1.00069370e-05, 1.00069370e-05, ...,
        1.00069371e-05, 1.00069370e-05, 1.00069371e-05],
       [1.00069370e-05, 1.00069370e-05, 1.00069370e-05, ...,
        1.00069371e-05, 1.00069370e-05, 1.00069371e-05],
       [1.00069370e-05, 1.00069370e-05, 1.00069370e-05, ...,
        1.00069371e-05, 1.00069370e-05, 1.00069371e-05],
       ...,
       [1.00069370e-05, 1.00069370e-05, 1.00069370e-05, ...,
        1.00069371e-05, 1.00069370e-05, 1.00069371e-05],
       [1.00069370e-05, 1.00069370e-05, 1.00069370e-05, ...,
        1.00069371e-05, 1.00069370e-05, 1.00069371e-05],
       [1.00069370e-05, 1.00069370e-05, 1.00069370e-05, ...,
        1.00069371e-05, 1.00069370e-05, 1.00069371e-05]])

In [45]:
y_final = np.zeros((10000,1))

In [47]:
y_final[0][0]

0.0

In [48]:
for i in range(10000):
  y_final[i][0] = np.argmax(ao1[i])

In [50]:
y_final.

AttributeError: ignored